# Agent Testing Notebook

This notebook is for testing the `LangGraphAgent` with different agent architectures.

## Setup

In [2]:
!pip install --upgrade pydantic

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv
load_dotenv()
from typing import Any, Callable, List, Optional, cast, Dict, Literal, Union
from pydantic import BaseModel, Field, field_validator
from langchain.tools import BaseTool, Tool

In [15]:
print(os.getenv("TAVILY_API_KEY"))

None


In [13]:
class SearchToolInput(BaseModel):
    query: str = Field(..., description="The search query to look up.")
    max_results: Optional[int] = Field(default=10, description="The maximum number of search results to return.")

# Define the Tool
class TavilySearchTool:
    def __init__(self, max_results: int = 10):
        self.max_results = max_results

    def search(self, query: str) -> Optional[List[Dict[str, Any]]]:
        """
        Perform a web search using the Tavily search engine.
        """
        try:
            # Initialize the Tavily search tool with the configured max_results
            search_tool = TavilySearchResults(max_results=self.max_results, tavily_api_key=os.getenv("TAVILY_API_KEY"))

            # Perform the search (synchronously)
            result = search_tool.invoke({"query": query})

            # Return the search results
            return result
        except Exception as e:
            return {"error": str(e)}

tavily_search_tool = Tool(
    name="Tavily Search",
    func=TavilySearchTool().search,
    description="Performs web searches using the Tavily search engine, providing accurate and trusted results for general queries.",
    args_schema=SearchToolInput
)


tavily_search_tool.invoke("when is the next nba season starting")

{'error': "1 validation error for TavilySearchAPIWrapper\n  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={'tavily_api_key': None}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.11/v/value_error"}

In [4]:
TavilySearchResults?

Init signature:
TavilySearchResults(
    *,
    name: str = 'tavily_search_results_json',
    description: str = 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
    args_schema: Type[pydantic.main.BaseModel] = <class 'langchain_community.tools.tavily_search.tool.TavilyInput'>,
    return_direct: bool = False,
    verbose: bool = False,
    callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    tags: Optional[list[str]] = None,
    metadata: Optional[dict[str, Any]] = None,
    handle_tool_error: Union[bool, str, Callable[[langchain_core.tools.base.ToolException], str], NoneType] = False,
    handle_validation_error: Union[bool, str, Callable[[Union[pydantic_core._pydantic_core.Va

## 1. Base Agent Test

In [ ]:
async def run_base_agent():
    print("--- Running Base Agent Test ---")
    session_id = str(uuid4())
    agent = LangGraphAgent(agent_type='base')
    
    messages = [Message(role="user", content="Hello! What is the weather in New York?")]
    
    response = await agent.get_response(messages, session_id)
    print(f"Session ID: {session_id}")
    print(f"Response: {response}")

# Run the async function
asyncio.run(run_base_agent())

## 2. Deep Agent Test

In [ ]:
async def run_deep_agent():
    print("
--- Running Deep Agent Test ---")
    session_id = str(uuid4())
    
    # This is a sample configuration for a deep agent.
    # You might need to adjust it based on the specifics of your `create_configurable_agent` function.
    deep_agent_config = {
        'default_instructions': 'You are a helpful assistant.',
        'default_sub_agents': [],
        'tools': []
    }
    
    agent = LangGraphAgent(agent_type='deepagent', agent_config=deep_agent_config)
    
    messages = [Message(role="user", content="Hello from the deep agent!")]
    
    response = await agent.get_response(messages, session_id)
    print(f"Session ID: {session_id}")
    print(f"Response: {response}")

# Run the async function
# asyncio.run(run_deep_agent()) # Uncomment to run

## 3. Supervisor Agent Test

In [ ]:
async def run_supervisor_agent():
    print("
--- Running Supervisor Agent Test ---")
    session_id = str(uuid4())

    # This is a sample configuration for a supervisor agent.
    # You will need to define your agents (e.g., using create_react_agent) and pass them in.
    supervisor_agent_config = {
        'agents': [
            # Example of a react agent config. You'd create the agent and pass it in.
            {
                'type': 'react',
                'model': 'gpt-4o',
                'tools': [],
                'name': 'researcher'
            }
        ],
        'supervisor_config': {
             # supervisor specific configs
        }
    }
    
    agent = LangGraphAgent(agent_type='supervisor', agent_config=supervisor_agent_config)
    
    messages = [Message(role="user", content="Hello from the supervisor agent!")]
    
    response = await agent.get_response(messages, session_id)
    print(f"Session ID: {session_id}")
    print(f"Response: {response}")

# Run the async function
# asyncio.run(run_supervisor_agent()) # Uncomment to run

## 4. Swarm Agent Test

In [ ]:
async def run_swarm_agent():
    print("
--- Running Swarm Agent Test ---")
    session_id = str(uuid4())

    # This is a sample configuration for a swarm agent.
    swarm_agent_config = {
        'agents': [
            {
                'type': 'react',
                'model': 'gpt-4o',
                'tools': [],
                'name': 'worker_bee_1'
            }
        ],
        'swarm_config': {
            'default_active_agent': 'worker_bee_1'
        }
    }
    
    agent = LangGraphAgent(agent_type='swarm', agent_config=swarm_agent_config)
    
    messages = [Message(role="user", content="Hello from the swarm agent!")]
    
    response = await agent.get_response(messages, session_id)
    print(f"Session ID: {session_id}")
    print(f"Response: {response}")

# Run the async function
# asyncio.run(run_swarm_agent()) # Uncomment to run